<a href="https://colab.research.google.com/github/tanishcode-12/assignment-1/blob/main/pathway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pathway
!pip install pathway bokeh geopy pandas
!pip install pathway bokeh pandas
!pip install -U pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 16.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 

In [29]:
import pathway
print(pathway.__version__)

0.24.1


In [30]:
import pathway as pw
import pandas as pd
from datetime import datetime
import time
import numpy as np
from geopy.distance import geodesic

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
output_notebook()

In [31]:
df = pd.read_csv('/content/dataset.csv')

# Combine date and time columns
df['Datetime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format="%d-%m-%Y %H:%M:%S")
df = df.sort_values(by='Datetime')

# Keep relevant columns
df = df[['Datetime', 'Occupancy', 'Capacity']]

In [32]:
df = pd.read_csv('/content/dataset.csv')

# Combine date and time columns
df['Datetime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format="%d-%m-%Y %H:%M:%S")
df = df.sort_values(by='Datetime')

# Keep relevant columns
df = df[['Datetime', 'Occupancy', 'Capacity']]

In [34]:
class ParkingSchema(pw.Schema):
    id: int
    datetime: DATE_TIME_NAIVE   # or pt.DATE_TIME_UTC
    occupancy: int
    capacity: int

In [35]:
def generate_csv_rows():
    for i, row in df.iterrows():
        yield {
            "id": i,
            "datetime": row['Datetime'],
            "occupancy": row['Occupancy'],
            "capacity": row['Capacity']
        }
        time.sleep(0.5)  # Simulate real-time stream (500ms delay per row)
        input_table = pw.io.python.read(generate_csv_rows, schema=ParkingSchema)


In [36]:

def baseline_price(i: int, occupancy: int, capacity: int, prev_price: float = 50.0) -> float:
    ratio = occupancy / capacity if capacity != 0 else 0
    return prev_price + 10 * ratio  # α = 10


In [37]:
def simulate_prices(df, alpha=10, initial_price=50):
    prices = [initial_price]
    for i in range(1, len(df)):
        occ = df.iloc[i]['Occupancy']
        cap = df.iloc[i]['Capacity']
        prev_price = prices[-1]
        ratio = occ / cap if cap else 0
        new_price = prev_price + alpha * ratio
        prices.append(new_price)
    df['PredictedPrice'] = prices
    return df

df = simulate_prices(df)


In [38]:
source = ColumnDataSource(df)

p = figure(x_axis_type='datetime', title="Baseline Linear Dynamic Pricing", width=800, height=400)
p.line(x='Datetime', y='PredictedPrice', source=source, legend_label='Predicted Price', line_width=2, color='navy')
p.circle(x='Datetime', y='PredictedPrice', source=source, size=8, color='orange', legend_label='Data Point')

hover = HoverTool(tooltips=[
    ("Time", "@Datetime{%F %T}"),
    ("Price", "@PredictedPrice{0.00}"),
    ("Occupancy", "@Occupancy"),
    ("Capacity", "@Capacity")
], formatters={'@Datetime': 'datetime'})

p.add_tools(hover)
p.xaxis.axis_label = 'Time'
p.yaxis.axis_label = 'Price'
p.legend.location = 'top_left'

show(p)


In [21]:
# STEP 2: Import libraries
import pathway as pw
import pandas as pd
import numpy as np
from pathway.internals.dtype import DATE_TIME_NAIVE
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from datetime import datetime
output_notebook()


In [22]:
# STEP 3: Load and prepare dataset
df = pd.read_csv('/content/dataset.csv')
df['Datetime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format="%d-%m-%Y %H:%M:%S")
df = df.sort_values(by='Datetime')


# Combine 'date' and 'time' if necessary, or parse directly
if 'date' in df.columns and 'time' in df.columns:
    df['Datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
elif 'datetime' in df.columns:
    df['Datetime'] = pd.to_datetime(df['datetime'])
else:
    df['Datetime'] = pd.to_datetime(df.iloc[:, 0])  # fallback

# Ensure all required columns exist and clean data
df['IsSpecialDay'] = df['IsSpecialDay'].astype(int)
df.fillna(0, inplace=True)

# Preview
df.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Datetime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,1970-01-01 00:00:00.000000000
5248,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,1970-01-01 00:00:00.000005248
3936,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,1970-01-01 00:00:00.000003936
6560,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,1970-01-01 00:00:00.000006560
17056,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,1970-01-01 00:00:00.000017056


In [23]:
# STEP 4: Define Pathway schema
class ParkingSchema(pw.Schema):
    id: int
    datetime: DATE_TIME_NAIVE
    occupancy: float
    capacity: float
    queue_length: float
    traffic_level: float
    is_special_day: int
    vehicle_weight: float


In [24]:
# STEP 5: Simulate stream from DataFrame
def simulate_stream():
    for _, row in df.iterrows():
        yield {
            "id": int(row.get("id", 0)),
            "datetime": row["Datetime"],
            "occupancy": float(row["Occupancy"]),
            "capacity": float(row["Capacity"]),
            "queue_length": float(row["QueueLength"]),
            "traffic_level": float(row["TrafficLevel"]),
            "is_special_day": int(row["IsSpecialDay"]),
            "vehicle_weight": float(row["VehicleWeight"])
        }

stream = pw.io.python.read(simulate_stream, schema=ParkingSchema)


TypeError: Function pathway.io.python.read() parameter subject=<function simulate_stream at 0x7fae14237d80> violates type hint <class 'pathway.io.python.ConnectorSubject'>, as <class "builtins.function"> <function simulate_stream at 0x7fae14237d80> not instance of <class "pathway.io.python.ConnectorSubject">.

In [25]:
# STEP 6: Define Demand & Price UDFs
@pw.udf
def compute_demand(occupancy, capacity, queue_length, traffic, is_special_day, vehicle_weight):
    α, β, γ, δ, ε = 1.0, 0.5, 0.3, 1.5, 0.8
    if capacity == 0:
        return 0.0
    return α * (occupancy / capacity) + β * queue_length - γ * traffic + δ * is_special_day + ε * vehicle_weight

@pw.udf
def normalize_demand(demand, min_d, max_d):
    if max_d == min_d:
        return 0
    return (demand - min_d) / (max_d - min_d)

@pw.udf
def compute_price(base_price, norm_demand):
    λ = 0.8
    price = base_price * (1 + λ * norm_demand)
    return max(min(price, base_price * 2), base_price * 0.5)


In [26]:
# STEP 7: Demand calculation
demand_table = stream.select(
    id=stream.id,
    datetime=stream.datetime,
    demand=compute_demand(
        stream.occupancy,
        stream.capacity,
        stream.queue_length,
        stream.traffic_level,
        stream.is_special_day,
        stream.vehicle_weight
    )
)

# STEP 8: Min/max demand values
min_demand = demand_table.demand.min()
max_demand = demand_table.demand.max()

# STEP 9: Normalize demand and compute price
base_price = 50  # static base price

result = demand_table.select(
    id=demand_table.id,
    datetime=demand_table.datetime,
    raw_demand=demand_table.demand,
    norm_demand=normalize_demand(demand_table.demand, min_demand, max_demand)
)

final = result.select(
    datetime=result.datetime,
    norm_demand=result.norm_demand,
    price=compute_price(base_price, result.norm_demand)
)

pw.io.python.write(final, name="FinalOutput")
pw.run()


NameError: name 'stream' is not defined

In [27]:
# STEP 10: Visualize results with Bokeh
final_df = pw.io.python.collect(final)
final_df = final_df.sort_values("datetime")

source = ColumnDataSource(data={
    'datetime': final_df['datetime'],
    'price': final_df['price'],
    'norm_demand': final_df['norm_demand']
})

p = figure(title="Demand-Based Dynamic Pricing", x_axis_type="datetime", width=800, height=400)
p.line('datetime', 'price', source=source, legend_label="Price", color="blue", line_width=2)
p.line('datetime', 'norm_demand', source=source, legend_label="Normalized Demand", color="red", line_width=2)

p.legend.location = "top_left"
p.xaxis.axis_label = "Datetime"
p.yaxis.axis_label = "Price / Normalized Demand"
show(p)


AttributeError: module 'pathway.io.python' has no attribute 'collect'

module 3


In [28]:
import pandas as pd
import numpy as np
import pathway as pw
from pathway.internals.dtype import DATE_TIME_NAIVE
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from geopy.distance import geodesic
from datetime import datetime
output_notebook()

# Step 3: Load dataset
df = pd.read_csv("/content/dataset.csv")
df['Datetime'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format="%d-%m-%Y %H:%M:%S")
df = df.sort_values(by='Datetime')

# Step 4: Define competitor lots
competitors = [
    {"name": "CompA", "lat": 26.145, "lon": 91.737, "price": 40},
    {"name": "CompB", "lat": 26.143, "lon": 91.734, "price": 55},
    {"name": "CompC", "lat": 26.147, "lon": 91.735, "price": 60}
]

# Step 5: Compute average competitor distance and pricing impact
def compute_price(row):
    base_price = row["base_price"]
    my_location = (row["lat"], row["lon"])
    occupancy_rate = row["occupancy"] / row["capacity"]

    distances = []
    cheaper_nearby = 0
    expensive_nearby = 0

    for comp in competitors:
        dist = geodesic(my_location, (comp["lat"], comp["lon"])).meters
        if dist < 500:  # only consider within 500m
            distances.append(dist)
            if comp["price"] < base_price:
                cheaper_nearby += 1
            else:
                expensive_nearby += 1

    # Pricing strategy:
    # ↓ Reduce price if full and cheaper options nearby
    # ↑ Raise price if nearby are expensive
    if occupancy_rate >= 1.0 and cheaper_nearby > 0:
        return base_price * 0.9
    elif expensive_nearby > 0:
        return base_price * 1.1
    else:
        return base_price

# Step 6: Create schema
class ParkingSchema(pw.Schema):
    id: int
    datetime: DATE_TIME_NAIVE
    occupancy: int
    capacity: int
    base_price: float
    lat: float
    lon: float

# Step 7: Simulate stream
def simulate_stream():
    for _, row in df.iterrows():
        yield {
            "id": row["id"],
            "datetime": row["Datetime"],
            "occupancy": row["occupancy"],
            "capacity": row["capacity"],
            "base_price": row["base_price"],
            "lat": row["lat"],
            "lon": row["lon"]
        }

stream = pw.io.python.read(simulate_stream, schema=ParkingSchema)

# Step 8: Apply competitive pricing
@pw.udf
def dynamic_price(occupancy, capacity, base_price, lat, lon):
    row = {
        "occupancy": occupancy,
        "capacity": capacity,
        "base_price": base_price,
        "lat": lat,
        "lon": lon
    }
    return compute_price(row)

result = stream.select(
    datetime=stream.datetime,
    adjusted_price=dynamic_price(
        stream.occupancy,
        stream.capacity,
        stream.base_price,
        stream.lat,
        stream.lon
    )
)

# Step 9: Write to Python and visualize with Bokeh
table = pw.io.python.write(result)

pw.run()

# Convert result to pandas DataFrame
data = table.to_pandas()
data.sort_values("datetime", inplace=True)

# Bokeh Visualization
source = ColumnDataSource(data)

p = figure(x_axis_type="datetime", title="Competitive Pricing Over Time", plot_height=350, plot_width=800)
p.line(x="datetime", y="adjusted_price", source=source, line_width=2, color="navy", legend_label="Price")
p.yaxis.axis_label = "Price"
p.xaxis.axis_label = "Time"
p.legend.location = "top_left"

show(p)


TypeError: Function pathway.io.python.read() parameter subject=<function simulate_stream at 0x7fae14235300> violates type hint <class 'pathway.io.python.ConnectorSubject'>, as <class "builtins.function"> <function simulate_stream at 0x7fae14235300> not instance of <class "pathway.io.python.ConnectorSubject">.